# Модель прогнозирования спроса
Краткое описание:
<br>Необходимо создать алгоритм прогноза спроса на 14 дней для товаров собственного
производства. Гранулярность ТК-SKU-День.

Задача:
<br>Построить модель прогноза спроса на основе мастер данных и данных продаж с учетом разных
признаков.

Метрика качества:
<br>`WAPE`:
<br>`def wape(y_true: np.array, y_pred: np.array):`
    <br>`return np.sum(np.abs(y_true-y_pred))/np.sum(np.abs(y_true))`

### Описание данных
1) sales_df_train.csv –данные по продажам за скользящий год для обучения.
    * Столбцы:
        - st_id – захэшированное id магазина;
        - pr_sku_id – захэшированное id товара;
        - date – дата;
        - pr_sales_type_id – флаг наличия промо;
        - pr_sales_in_units – число проданных товаров без признака промо;
        - pr_promo_sales_in_units – число проданных товаров с признаком промо;
        - pr_sales_in_rub – продажи без признака промо в РУБ;
        - pr_promo_sales_in_rub – продажи с признаком промо в РУБ;
2) pr_df.csv – данные по товарной иерархии.
<br>От большего к меньшему pr_group_id - pr_cat_id - pr_subcat_id - pr_sku_id.
    - Столбцы:
        - pr_group_id – захэшированная группа товара;
        - pr_cat_id – захэшированная категория товара;
        - pr_subcat_id – захэшированная подкатегория товара;
        - pr_sku_id – захэшированное id товара;
        - pr_uom_id (маркер, обозначающий продаётся товар на вес или в ШТ).
3) pr_st.csv – данные по магазинам.
    - Столбцы:
        - st_id – захэшированное id магазина;
        - st_city_id – захэшированное id города;
        - st_division_code id – захэшированное id дивизиона;
        - st_type_format_id – id формата магазина;
        - st_type_loc_id – id тип локации/окружения магазина;
        - st_type_size_id – id типа размера магазина;
        - st_is_active – флаг активного магазина на данный момент.

## Загрузка библиотек

In [1]:
import pandas as pd
import numpy as np

## Знакомство с данными

In [10]:
sales_df_train = pd.read_csv('sp_sales_task/sales_df_train.csv')
pr_df = pd.read_csv('sp_sales_task/pr_df.csv')
pr_st = pd.read_csv('sp_sales_task/st_df.csv')

# sales_df_train.head()
# pr_df.head()
# pr_st.head()

Пример файла с результатом прогноза

In [9]:
sales_submission_ex = pd.read_csv('sp_sales_task/sales_submission.csv')
sales_submission_ex.head()

,st_id,pr_sku_id,date,target
0,16a5cdae362b8d27a1d8f8c7b78b4330,0045ebdb1069ff4b3dd3efe628c39cd3,2023-07-20,0
1,16a5cdae362b8d27a1d8f8c7b78b4330,00661699f543753ec7e911a64b9fd2f6,2023-07-20,0
2,16a5cdae362b8d27a1d8f8c7b78b4330,0094042bfeae507dc7f62acc8e5ed03a,2023-07-20,0
3,16a5cdae362b8d27a1d8f8c7b78b4330,0169529ff660adcac9b7e354e0c4b882,2023-07-20,0
4,16a5cdae362b8d27a1d8f8c7b78b4330,01e4734745e97e52d3213449e1a05dd7,2023-07-20,0


In [19]:
display(sales_df_train.sample(1))
sales_df_train.info()
display(pr_df.sample(1))
pr_df.info()
display(pr_st.sample(1))
pr_st.info()

,st_id,pr_sku_id,date,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub
495666,6364d3f0f495b6ab9dcf8d3b5c6e0b01,1a0f36f6e92560bcae205b6c32cc033f,2023-03-06,0,7.0,0.0,313.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883015 entries, 0 to 883014
Data columns (total 8 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   date                     883015 non-null  object 
 3   pr_sales_type_id         883015 non-null  int64  
 4   pr_sales_in_units        883015 non-null  float64
 5   pr_promo_sales_in_units  883015 non-null  float64
 6   pr_sales_in_rub          883015 non-null  float64
 7   pr_promo_sales_in_rub    883015 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 53.9+ MB


,pr_sku_id,pr_group_id,pr_cat_id,pr_subcat_id,pr_uom_id
644,21716a96b18e53b7f324cae2ee59d405,c74d97b01eae257e44aa9d5bade97baf,c559da2ba967eb820766939a658022c8,89b0107c6d0aca5f15fa7a715f9e06e5,17


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2050 entries, 0 to 2049
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   pr_sku_id     2050 non-null   object
 1   pr_group_id   2050 non-null   object
 2   pr_cat_id     2050 non-null   object
 3   pr_subcat_id  2050 non-null   object
 4   pr_uom_id     2050 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 80.2+ KB


,st_id,st_city_id,st_division_code,st_type_format_id,st_type_loc_id,st_type_size_id,st_is_active
8,fa7cdfad1a5aaf8370ebeda47a1ff1c3,885fe656777008c335ac96072a45be15,296bd0cc6e735f9d7488ebc8fbc19130,1,1,12,1


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   st_id              12 non-null     object
 1   st_city_id         12 non-null     object
 2   st_division_code   12 non-null     object
 3   st_type_format_id  12 non-null     int64 
 4   st_type_loc_id     12 non-null     int64 
 5   st_type_size_id    12 non-null     int64 
 6   st_is_active       12 non-null     int64 
dtypes: int64(4), object(3)
memory usage: 800.0+ bytes


### Открытие файла с распознаванием дат и формированием новых индексов

In [22]:
sales_dt = pd.read_csv('sp_sales_task/sales_df_train.csv', index_col = [2], parse_dates = [2])
sales_dt.sort_index(inplace=True)
sales_dt.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 883015 entries, 2022-08-01 to 2023-07-18
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   st_id                    883015 non-null  object 
 1   pr_sku_id                883015 non-null  object 
 2   pr_sales_type_id         883015 non-null  int64  
 3   pr_sales_in_units        883015 non-null  float64
 4   pr_promo_sales_in_units  883015 non-null  float64
 5   pr_sales_in_rub          883015 non-null  float64
 6   pr_promo_sales_in_rub    883015 non-null  float64
dtypes: float64(4), int64(1), object(2)
memory usage: 53.9+ MB


,st_id,pr_sku_id,pr_sales_type_id,pr_sales_in_units,pr_promo_sales_in_units,pr_sales_in_rub,pr_promo_sales_in_rub
date,,,,,,,
2022-08-01,42a0e188f5033bc65bf8d78622277c4e,5781a2637b476d781eb3134581b32044,0,2.0,0.0,463.0,0.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,82704fa1c2dbdb928bf4eed0667260dd,0,2.0,0.0,204.0,0.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,0cc8850f66397af21700e3c060a5210c,0,9.0,0.0,639.0,0.0
2022-08-01,fa7cdfad1a5aaf8370ebeda47a1ff1c3,c4a665596d4f67cecb7542c9fad407ee,0,13.0,0.0,1734.0,0.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,dce1f234d6424aa61f8e7ce0baffd9af,0,2.0,0.0,313.0,0.0
2022-08-01,f7e6c85504ce6e82442c770f7c8606f0,b43e19936412e99b8efd1f9061d3f3c7,0,5.0,0.0,185.0,0.0
2022-08-01,c81e728d9d4c2f636f067f89cc14862c,282984b385d80166ba0d4f32a7843f53,0,3.0,0.0,459.0,0.0
2022-08-01,f7e6c85504ce6e82442c770f7c8606f0,9029365b9b6fc2800431fa6f2be03569,0,4.0,0.0,366.0,0.0
2022-08-01,fa7cdfad1a5aaf8370ebeda47a1ff1c3,dc8e1d0c7e95ae832902953b1dc4a22d,0,3.0,0.0,735.0,0.0


### Предобработка данных

In [24]:
#sales_dt['share_pr_sales'] = 

array([0, 1])

### Выводы
- Пропусков в данных не обнаружено
- Нужно заменить тип данных в поле `date` на `datetime`

## Чек-лист
1. Файл в зафиксированном формате с результатом прогноза спроса (sales_submission.csv).
2. Воспроизводимый код на Python
3. Описание решения:
    
    a. Описание обученной модели прогноза спроса
    
        i. Признаки
        ii. интерпретация (shapley values),
        iii. кросс-валидация
        iv. алгоритмы
    
    b. Описание вашего алгоритма оптимизации:
    
        i. методология расчетов
        ii. скорость оптимизации